<a href="https://colab.research.google.com/github/Hidestament/AtCoder/blob/main/ABC/ABC213.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [AtCoder Beginner Contest 213](https://atcoder.jp/contests/abc213)

## [A - Bitwise Exclusive Or](https://atcoder.jp/contests/abc213/tasks/abc213_a)

In [ ]:
A, B = map(int, input().split())
print(A ^ B)

## [B - Booby Prize](https://atcoder.jp/contests/abc213/tasks/abc213_b)

In [ ]:
N = int(input())
A = list(map(int, input().split()))

enumerate_A = [[i+1, A[i]] for i in range(N)]
enumerate_A.sort(key=lambda x:x[1])
print(enumerate_A[-2][0])

6
1 123 12345 12 1234 123456
5


## [C - Reorder Cards](https://atcoder.jp/contests/abc213/tasks/abc213_c)

In [2]:
H, W, N = map(int, input().split())
hgrid = []
wgrid = []
for _ in range(N):
    a, b = map(int, input().split())
    hgrid.append(a)
    wgrid.append(b)

4 5 2
3 2
2 5


In [6]:
hcompress = {x: i for i, x in enumerate(sorted(set(hgrid)), start=1)}
wcompress = {x: i for i, x in enumerate(sorted(set(wgrid)), start=1)}

In [8]:
for h, w in zip(hgrid, wgrid):
    print(hcompress[h], wcompress[w])

2 1
1 2


## [D - Takahashi Tour](https://atcoder.jp/contests/abc213/tasks/abc213_d)

In [ ]:
import sys
sys.setrecursionlimit(10**7)

N = int(input())
graph = [[] for _ in range(N)]
for _ in range(N-1):
    a, b = map(int, input().split())
    graph[a-1].append(b-1)
    graph[b-1].append(a-1)
for i in range(N):
    graph[i] = sorted(graph[i])

4
1 2
4 2
3 1


In [ ]:
euler_tour = []
def dfs(now, bef=-1):
    euler_tour.append(now+1)
    for to in graph[now]:
        if to != bef:
            dfs(to, now)
            euler_tour.append(now+1)

In [ ]:
dfs(0)

In [ ]:
print(*euler_tour)

[1, 2, 4, 2, 1, 3, 1]

オイラーツアーをライブラリ化しちゃおう

In [ ]:
import sys
sys.setrecursionlimit(10**7)

N = int(input())
graph = [[] for _ in range(N)]
for _ in range(N-1):
    a, b = map(int, input().split())
    graph[a-1].append(b-1)
    graph[b-1].append(a-1)
for i in range(N):
    graph[i] = sorted(graph[i])

In [ ]:
import sys
sys.setrecursionlimit(10**7)


def make_euler_tour(graph, r):
    euler_tour = []

    def dfs(now, bef=-1):
        nonlocal euler_tour
        # 初めて尋ねた瞬間
        euler_tour.append(now)
        for to in graph[now]:
            if to != bef:
                dfs(to, now)
                # nowの子を全て訪問し, nowに戻ってきた瞬間
                euler_tour.append(now)
    dfs(r)
    return euler_tour

In [ ]:
euler_tour = make_euler_tour(graph, 0)
for i in euler_tour:
    print(i + 1)

## [E - Stronger Takahashi](https://atcoder.jp/contests/abc213/tasks/abc213_e)

In [ ]:
H, W = map(int, input().split())
grid = [input() for _ in range(H)]

In [ ]:
from collections import deque
dq = deque()
dq.append([0, 0])
dist = [[10**10] * W for _ in range(H)]
dist[0][0] = 0

while dq:
    h, w = dq.popleft()
    # コスト0の移動
    for dh, dw in zip([1, 0, -1, 0], [0, 1, 0, -1]):
        nh, nw = h + dh, w + dw
        if (0 <= nh < H) and (0 <= nw < W):
            if grid[nh][nw] == "#": continue
            if dist[nh][nw] > dist[h][w]:
                dist[nh][nw] = dist[h][w]
                dq.appendleft([nh, nw])
    
    # コスト1の移動
    for dh in range(-3, 3):
        for dw in range(-3, 3):
            nh, nw = h + dh, w + dw
            if abs(h - nh) + abs(w - nw) <= 3:
                nh, nw = h + dh, w + dw
                if (0 <= nh < H) and (0 <= nw < W):
                    if dist[nh][nw] > dist[h][w] + 1:
                        dq.append([nh, nw])
                        dist[nh][nw] = dist[h][w] + 1

In [ ]:
print(dist[-1][-1])